In [1]:
import warnings
#os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [2]:
import sys
import os
import gc
import re
import bisect

import pandas as pd
import polars as pl
import numpy as np
import pyarrow.parquet as pq

import sklearn.metrics as m
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer, \
                                  OneHotEncoder, OrdinalEncoder, Normalizer

from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.ensemble import StackingClassifier 
from sklearn.pipeline import Pipeline, FeatureUnion
from lightgbm import LGBMClassifier


In [3]:
def url_transformer(url):
  
    url = ' '.join(set(url))
    
    return url

In [4]:
def gini(y_true, y_pred_prob):
    return (2 * m.roc_auc_score(y_true, y_pred_prob) - 1)

In [5]:
LOCAL_DATA_PATH = 'D:\\ods_ai'
SPLIT_SEED = 42
DATA_FILE = 'dataset_full.parquet'
TARGET_FILE = 'public_train.pqt'
SUBMISSION_FILE = 'submit_2.pqt'

In [6]:
%%time
data = pl.read_parquet(f'{LOCAL_DATA_PATH}\\{DATA_FILE}')

CPU times: total: 57 s
Wall time: 1min 6s


In [7]:
data.shape

(322899435, 12)

In [8]:
data.head()

region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
cat,cat,cat,cat,cat,cat,cat,f32,datetime[ns],cat,i8,i32
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""ad.adriver.ru""","""smartphone""","""iOS""",20368.0,2022-06-15 00:00:00,"""morning""",1,45098
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""apple.com""","""smartphone""","""iOS""",20368.0,2022-06-19 00:00:00,"""morning""",1,45098
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""avatars.mds.ya...","""smartphone""","""iOS""",20368.0,2022-06-12 00:00:00,"""day""",1,45098
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""googleads.g.do...","""smartphone""","""iOS""",20368.0,2022-05-16 00:00:00,"""day""",1,45098
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""googleads.g.do...","""smartphone""","""iOS""",20368.0,2022-05-30 00:00:00,"""day""",1,45098


In [9]:
%%time
#время суток и день недели действия пользователя
data = (data.with_columns(pl.col('date').dt.weekday().alias('weekday'))
            .with_columns((pl.col('part_of_day') + '_' + pl.col('weekday').cast(pl.Utf8)).alias('wd')))

CPU times: total: 39 s
Wall time: 1min 42s


In [10]:
data.head()

region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,weekday,wd
cat,cat,cat,cat,cat,cat,cat,f32,datetime[ns],cat,i8,i32,u32,str
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""ad.adriver.ru""","""smartphone""","""iOS""",20368.0,2022-06-15 00:00:00,"""morning""",1,45098,3,"""morning_3"""
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""apple.com""","""smartphone""","""iOS""",20368.0,2022-06-19 00:00:00,"""morning""",1,45098,7,"""morning_7"""
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""avatars.mds.ya...","""smartphone""","""iOS""",20368.0,2022-06-12 00:00:00,"""day""",1,45098,7,"""day_7"""
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""googleads.g.do...","""smartphone""","""iOS""",20368.0,2022-05-16 00:00:00,"""day""",1,45098,1,"""day_1"""
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""googleads.g.do...","""smartphone""","""iOS""",20368.0,2022-05-30 00:00:00,"""day""",1,45098,1,"""day_1"""


In [11]:
%%time
data_tf = data.groupby('user_id').agg(pl.col('url_host'))

CPU times: total: 1.61 s
Wall time: 7.53 s


In [12]:
data_tf.head()

user_id,url_host
i32,list[cat]
182144,"[""api.facemojikeyboard.com"", ""i.ytimg.com"", ... ""mail.yandex.ru""]"
303152,"[""ad.adriver.ru"", ""ad.mail.ru"", ... ""zen.yandex.ru""]"
256752,"[""ok.ru"", ""online.sberbank.ru"", ... ""online.sberbank.ru""]"
326560,"[""ad.mail.ru"", ""googleads.g.doubleclick.net"", ... ""yastatic.net""]"
409200,"[""ad.adriver.ru"", ""playep.pro"", ... ""vk.com""]"


In [13]:
data.null_count()

region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,weekday,wd
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,6589482,0,0,0,0,0,0


# Feature engineering

In [14]:
add_features = data.select([pl.col('user_id').unique()])

In [15]:
add_features.shape

(415317, 1)

In [16]:
day_part_list = data.select([pl.col('part_of_day').unique()]).to_series().to_list()

In [17]:
%%time
#количество запросов по времени суток
for day_part in day_part_list:
    add_features = add_features.join(data.select(['part_of_day', 'request_cnt', 'user_id']) \
                                         .filter(pl.col('part_of_day')==day_part) \
                                         .groupby('user_id') \
                                         .agg(pl.col('request_cnt').sum().alias(f'request_cnt_{day_part}')), 
                                      how='left', on='user_id')

add_features.shape

CPU times: total: 19.7 s
Wall time: 16.7 s


(415317, 5)

In [18]:
add_features.head()

user_id,request_cnt_day,request_cnt_evening,request_cnt_morning,request_cnt_night
i32,i64,i64,i64,i64
0,107,62,23,1
1,363,309,338,37
2,198,130,77,6
3,97,125,49,4
4,271,206,289,11


In [19]:
%%time
#модель телефона
add_features = add_features.join(data.select(['cpe_model_name', 'user_id']) \
                                     .groupby('user_id') \
                                     .agg(pl.col('cpe_model_name').first()), 
                                 how='left', on='user_id')

CPU times: total: 12.4 s
Wall time: 2.08 s


In [20]:
%%time
#город с наибольшим количеством запросов
add_features = add_features.join(data.select(['city_name', 'user_id', 'request_cnt']) \
                                     .groupby('user_id', 'city_name') \
                                     .agg(pl.col('request_cnt').sum())
                                     .groupby('user_id')
                                     .agg(pl.col('city_name').sort_by('request_cnt', descending=True).first()), 
                                 how='left', on='user_id')

CPU times: total: 1min 16s
Wall time: 8.31 s


In [21]:
%%time
#регион с наибольшим количеством запросов
add_features = add_features.join(data.select(['region_name', 'user_id', 'request_cnt']) \
                                     .groupby('user_id', 'region_name') \
                                     .agg(pl.col('request_cnt').sum())
                                     .groupby('user_id')
                                     .agg(pl.col('region_name').sort_by('request_cnt', descending=True).first()), 
                                 how='left', on='user_id')

CPU times: total: 1min
Wall time: 7.16 s


In [22]:
%%time
#тип устройства
add_features = add_features.join(data.select(['cpe_type_cd', 'user_id']) \
                                      .groupby('user_id') \
                                      .agg(pl.col('cpe_type_cd').first()), 
                                 how='left', on='user_id')

CPU times: total: 12.9 s
Wall time: 2.08 s


In [23]:
%%time
#максимальное количество запросов по времени суток
for day_part in day_part_list:
    add_features = add_features.join(data.select(['part_of_day', 'user_id', 'request_cnt']) 
                                         .filter(pl.col('part_of_day')==day_part)
                                         .groupby('user_id')
                                         .agg(pl.col('request_cnt').max().alias(f'request_cnt_max_{day_part}')),
                                     how='left', on='user_id')

add_features.shape

CPU times: total: 20.5 s
Wall time: 4.44 s


(415317, 13)

In [24]:
%%time
#максимальное количество запросов по url
add_features = add_features.join((data.select(['url_host', 'user_id', 'request_cnt']) \
                                      .groupby('user_id', 'url_host') \
                                      .agg(pl.col('request_cnt').sum().alias('max_per_url'))
                                      .groupby('user_id')
                                      .agg(pl.col('max_per_url').max())), 
                                 how='left', on='user_id')

CPU times: total: 3min 30s
Wall time: 25.4 s


In [25]:
%%time
#максимальное количество запросов за день
add_features = add_features.join(data.select(['date', 'user_id', 'request_cnt']) \
                                     .groupby('user_id', 'date') \
                                     .agg(pl.col('request_cnt').sum().alias('max_per_day'))
                                     .groupby('user_id')
                                     .agg(pl.col('max_per_day').max()), 
                                 how='left', on='user_id')

CPU times: total: 1min 24s
Wall time: 9.12 s


In [26]:
%%time
#суммарное количество запросов
add_features = add_features.with_columns([(pl.col('request_cnt_day') + 
                                           pl.col('request_cnt_evening') +
                                           pl.col('request_cnt_morning') +
                                           pl.col('request_cnt_night')).alias('total_cnt')
                                          ])

CPU times: total: 0 ns
Wall time: 42.3 ms


In [27]:
%%time
#производитель устройства
add_features = add_features.join(data.select(['cpe_manufacturer_name', 'user_id']) \
                                     .groupby('user_id') \
                                     .agg(pl.col('cpe_manufacturer_name').first()), 
                                 how='left', on='user_id')

CPU times: total: 12.5 s
Wall time: 2.05 s


In [28]:
%%time
#ос устройства
add_features = add_features.join(data.select(['cpe_model_os_type', 'user_id']) \
                                     .groupby('user_id') \
                                     .agg(pl.col('cpe_model_os_type').first()), 
                                 how='left', on='user_id')

CPU times: total: 13.4 s
Wall time: 2.03 s


In [29]:
%%time
#количество уникальных регионов
add_features = add_features.join(data.select(['region_name','user_id']) \
                                     .groupby('user_id') \
                                     .agg(pl.col('region_name').n_unique().alias('region_name_count')), 
                                 how='left', on='user_id')

CPU times: total: 18.7 s
Wall time: 1.51 s


In [30]:
%%time
#количество уникальных городов
add_features = add_features.join(data.select(['city_name','user_id']) \
                                     .groupby('user_id') \
                                     .agg(pl.col('city_name').n_unique().alias('city_name_count')), 
                                 how='left', on='user_id')

CPU times: total: 37.2 s
Wall time: 5.87 s


In [31]:
%%time
#количество уникальных url
add_features = add_features.join(data.select(['url_host','user_id']) \
                                     .groupby('user_id') \
                                     .agg(pl.col('url_host').n_unique().alias('url_host_count')), 
                                 how='left', on='user_id')

CPU times: total: 22.8 s
Wall time: 2.66 s


In [32]:
%%time
#средняя цена устройства
add_features = add_features.join(data.select(['price','user_id']) \
                                     .groupby('user_id') \
                                     .agg(pl.col('price').mean()), 
                                 how='left', on='user_id')

CPU times: total: 48.5 s
Wall time: 9.04 s


In [33]:
add_features.head()

user_id,request_cnt_day,request_cnt_evening,request_cnt_morning,request_cnt_night,cpe_model_name,city_name,region_name,cpe_type_cd,request_cnt_max_day,request_cnt_max_evening,request_cnt_max_morning,request_cnt_max_night,max_per_url,max_per_day,total_cnt,cpe_manufacturer_name,cpe_model_os_type,region_name_count,city_name_count,url_host_count,price
i32,i64,i64,i64,i64,cat,cat,cat,cat,i8,i8,i8,i8,i64,i64,i64,cat,cat,u32,u32,u32,f32
0,107,62,23,1,"""Galaxy J1 2016...","""Москва""","""Москва""","""smartphone""",5,3,2,1,38,82,193,"""Samsung""","""Android""",1,1,52,2990.0
1,363,309,338,37,"""Mi 9""","""Москва""","""Москва""","""smartphone""",6,5,5,3,144,102,1047,"""Xiaomi""","""Android""",3,6,136,null
2,198,130,77,6,"""Honor 9 Lite""","""Печора""","""Республика Ком...","""smartphone""",4,4,3,1,58,31,411,"""Huawei""","""Android""",1,1,51,5915.0
3,97,125,49,4,"""P Smart 2021""","""Воронеж""","""Воронежская об...","""smartphone""",5,5,2,2,49,34,275,"""Huawei Device ...","""Android""",1,1,43,13990.0
4,271,206,289,11,"""Nova 3""","""Анапа""","""Краснодарский ...","""smartphone""",5,4,4,2,65,107,777,"""Huawei""","""Android""",5,9,108,12990.0


In [34]:
add_features.null_count()

user_id,request_cnt_day,request_cnt_evening,request_cnt_morning,request_cnt_night,cpe_model_name,city_name,region_name,cpe_type_cd,request_cnt_max_day,request_cnt_max_evening,request_cnt_max_morning,request_cnt_max_night,max_per_url,max_per_day,total_cnt,cpe_manufacturer_name,cpe_model_os_type,region_name_count,city_name_count,url_host_count,price
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,11608,25146,20757,77471,0,0,0,0,11608,25146,20757,77471,0,0,84888,0,0,0,0,0,10703


In [35]:
add_features = add_features.with_columns(pl.col('price').fill_null(strategy='mean'))

In [36]:
%%time
add_features = add_features.with_columns(pl.col('request_cnt_day').fill_null(value=0),
                                         pl.col('request_cnt_evening').fill_null(value=0),
                                         pl.col('request_cnt_morning').fill_null(value=0),
                                         pl.col('request_cnt_night').fill_null(value=0),
                                         pl.col('total_cnt').fill_null(value=0))

CPU times: total: 0 ns
Wall time: 62.5 ms


In [37]:
%%time
add_features = add_features.with_columns(pl.col('request_cnt_max_day').fill_null(value=0),
                                         pl.col('request_cnt_max_evening').fill_null(value=0),
                                         pl.col('request_cnt_max_morning').fill_null(value=0),
                                         pl.col('request_cnt_max_night').fill_null(value=0))

CPU times: total: 0 ns
Wall time: 24.6 ms


In [38]:
add_features.null_count()

user_id,request_cnt_day,request_cnt_evening,request_cnt_morning,request_cnt_night,cpe_model_name,city_name,region_name,cpe_type_cd,request_cnt_max_day,request_cnt_max_evening,request_cnt_max_morning,request_cnt_max_night,max_per_url,max_per_day,total_cnt,cpe_manufacturer_name,cpe_model_os_type,region_name_count,city_name_count,url_host_count,price
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
%%time
#список 2500 самых популярных url
top_2500 = (data.select('url_host', 'request_cnt')
                .groupby('url_host')
                .agg(pl.col('request_cnt').sum())
                .sort('request_cnt', descending=True)
                .limit(2500)).select('url_host').to_pandas()['url_host'].to_list()

CPU times: total: 35 s
Wall time: 6.49 s


In [40]:
%%time
#2500 самых популярных url
add_features = add_features.join(data.select('user_id', 'request_cnt', 'url_host')
                                     .filter(pl.col('url_host').cast(pl.Utf8).is_in(top_2500))
                                     .pivot(values='request_cnt', 
                                            index='user_id', 
                                            columns='url_host', 
                                            aggregate_function='sum'),
                                 how='left', on='user_id')

CPU times: total: 5min 26s
Wall time: 1min 58s


In [41]:
add_features.null_count()

user_id,request_cnt_day,request_cnt_evening,request_cnt_morning,request_cnt_night,cpe_model_name,city_name,region_name,cpe_type_cd,request_cnt_max_day,request_cnt_max_evening,request_cnt_max_morning,request_cnt_max_night,max_per_url,max_per_day,total_cnt,cpe_manufacturer_name,cpe_model_os_type,region_name_count,city_name_count,url_host_count,price,ad.adriver.ru,apple.com,avatars.mds.yandex.net,googleads.g.doubleclick.net,i.ytimg.com,icloud.com,m.avito.ru,relap.io,sun9-5.userapi.com,ads.adfox.ru,banners.adfox.ru,mail.yandex.ru,syndication.realsrv.com,ad.mail.ru,sun9-77.userapi.com,...,my.games,serialmix.ru,animevost.org,svetofor-magazin.su,gdedps.com,poisk123.ru,ww1.survey-smiles.com,portal.mi-room.ru,off.daily-inform.ru,oksmi.site,avalon-3.antorgps.com,hitmo.org,w1.dwar.ru,besposrednika.ru,maximum.ru,fclmnews.ru,fanat1k.ru,ognyvo.ru,saas.navixy.com,v2.vost.pw,anybalance.ru,ya-turbo.ru,sverhestestvennoe.club,hawk.live,dh-online.ru,seasonhit.tk,gotovim-online.ru,w3m.huawei.com,huawei.fotostrana.ru,news-show.online,hmn.ru,ladys-room.ru,info.flyredwings.com,wap1.3101010.ru,ranobes.com,d3.taxinonstop.ru,avalon.antorgps.com
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,...,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123749,271678,32625,20755,34049,316465,274203,173510,189399,83713,219851,180631,324042,40579,208872,...,415174,415237,415213,415262,415210,415150,415104,415237,415285,415172,415210,415224,415257,415098,415125,415189,414992,415269,415242,415255,415266,415209,415257,415201,415174,415240,415283,415290,415222,415207,415306,415265,415293,415282,415201,415267,415222


In [42]:
add_features = add_features.with_columns(pl.col(top_2500).fill_null(value=0))

In [43]:
add_features.head()

user_id,request_cnt_day,request_cnt_evening,request_cnt_morning,request_cnt_night,cpe_model_name,city_name,region_name,cpe_type_cd,request_cnt_max_day,request_cnt_max_evening,request_cnt_max_morning,request_cnt_max_night,max_per_url,max_per_day,total_cnt,cpe_manufacturer_name,cpe_model_os_type,region_name_count,city_name_count,url_host_count,price,ad.adriver.ru,apple.com,avatars.mds.yandex.net,googleads.g.doubleclick.net,i.ytimg.com,icloud.com,m.avito.ru,relap.io,sun9-5.userapi.com,ads.adfox.ru,banners.adfox.ru,mail.yandex.ru,syndication.realsrv.com,ad.mail.ru,sun9-77.userapi.com,...,my.games,serialmix.ru,animevost.org,svetofor-magazin.su,gdedps.com,poisk123.ru,ww1.survey-smiles.com,portal.mi-room.ru,off.daily-inform.ru,oksmi.site,avalon-3.antorgps.com,hitmo.org,w1.dwar.ru,besposrednika.ru,maximum.ru,fclmnews.ru,fanat1k.ru,ognyvo.ru,saas.navixy.com,v2.vost.pw,anybalance.ru,ya-turbo.ru,sverhestestvennoe.club,hawk.live,dh-online.ru,seasonhit.tk,gotovim-online.ru,w3m.huawei.com,huawei.fotostrana.ru,news-show.online,hmn.ru,ladys-room.ru,info.flyredwings.com,wap1.3101010.ru,ranobes.com,d3.taxinonstop.ru,avalon.antorgps.com
i32,i64,i64,i64,i64,cat,cat,cat,cat,i8,i8,i8,i8,i64,i64,i64,cat,cat,u32,u32,u32,f32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,...,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,107,62,23,1,"""Galaxy J1 2016...","""Москва""","""Москва""","""smartphone""",5,3,2,1,38,82,193,"""Samsung""","""Android""",1,1,52,2990.0,1,0,9,38,5,0,4,2,0,5,2,0,0,6,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,363,309,338,37,"""Mi 9""","""Москва""","""Москва""","""smartphone""",6,5,5,3,144,102,1047,"""Xiaomi""","""Android""",3,6,136,28749.511719,7,0,109,76,42,0,0,6,0,20,2,144,0,36,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,198,130,77,6,"""Honor 9 Lite""","""Печора""","""Республика Ком...","""smartphone""",4,4,3,1,58,31,411,"""Huawei""","""Android""",1,1,51,5915.0,0,0,30,38,52,0,0,0,2,11,0,0,0,16,4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,97,125,49,4,"""P Smart 2021""","""Воронеж""","""Воронежская об...","""smartphone""",5,5,2,2,49,34,275,"""Huawei Device ...","""Android""",1,1,43,13990.0,2,0,26,6,2,0,0,2,1,0,0,14,0,22,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,271,206,289,11,"""Nova 3""","""Анапа""","""Краснодарский ...","""smartphone""",5,4,4,2,65,107,777,"""Huawei""","""Android""",5,9,108,12990.0,7,0,36,48,38,0,4,10,10,22,7,0,0,48,4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
wd_list = data.select(pl.col('wd').unique()).to_pandas()['wd'].to_list()

In [45]:
%%time
#количество запросов по времени суток и дню недели действия пользователя
add_features = add_features.join(data.pivot(values='request_cnt', 
                                            index='user_id', 
                                            columns='wd', 
                                            aggregate_function='sum'),
                                 how='left', on='user_id')

CPU times: total: 2min 14s
Wall time: 49.1 s


In [46]:
add_features = add_features.with_columns(pl.col(wd_list).fill_null(value=0))

In [47]:
add_features = add_features.rename({keys: keys + '_domain' for keys in dict.fromkeys(top_2500).keys()})

In [48]:
top_2500 = [keys + '_domain' for keys in dict.fromkeys(top_2500).keys()]

In [49]:
num_features_list = ['request_cnt_morning', 'request_cnt_day', 'request_cnt_evening',
                     'request_cnt_night', 'price', 'url_host_count', 'city_name_count', 
                     'region_name_count','request_cnt_max_day', 'request_cnt_max_evening',
                     'request_cnt_max_morning', 'request_cnt_max_night', 'total_cnt',
                     'max_per_url', 'max_per_day']

In [50]:
cat_features_list = ['cpe_model_name', 'city_name', 'cpe_type_cd', 'region_name', 'cpe_model_os_type', 'cpe_manufacturer_name']

In [51]:
add_features = add_features.to_pandas()

In [52]:
data_tf = data_tf.to_pandas()

In [53]:
data_tf.head()

,user_id,url_host
0,182144,"[api.facemojikeyboard.com, i.ytimg.com, unbloc..."
1,303152,"[ad.adriver.ru, ad.mail.ru, avatars.mds.yandex..."
2,256752,"[ok.ru, online.sberbank.ru, cdn.viqeo.tv, vk.c..."
3,326560,"[ad.mail.ru, googleads.g.doubleclick.net, goog..."
4,409200,"[ad.adriver.ru, playep.pro, static.skwstat.ru,..."


In [54]:
del data
gc.collect()

0

# Prepare data

In [55]:
targets = pq.read_table(f'{LOCAL_DATA_PATH}\\{TARGET_FILE}')
pd.DataFrame([(z.name, z.type) for z in targets.schema], columns = [['field', 'type']])

,field,type
0,age,double
1,is_male,string
2,user_id,int64
3,__index_level_0__,int64


In [56]:
usr_targets = targets.to_pandas()
usr_targets.dropna(inplace=True)
usr_targets = usr_targets[usr_targets['is_male'] != 'NA']
usr_targets['is_male'] = usr_targets['is_male'].astype(int)
df = usr_targets.merge(data_tf, how = 'inner', on = ['user_id'])
df = df.merge(add_features, how = 'inner', on = ['user_id'])
df.head()

,age,is_male,user_id,url_host,request_cnt_day,request_cnt_evening,request_cnt_morning,request_cnt_night,cpe_model_name,city_name,...,night_6,morning_6,evening_6,evening_5,morning_5,day_2,evening_7,day_5,night_1,night_7
0,31.0,1,350459,"[sun9-29.userapi.com, id.tinkoff.ru, id.tinkof...",122,30,88,0,Redmi Note 7,Ижевск,...,0,2,3,8,7,17,0,22,0,0
1,35.0,1,188276,"[onlayn-radio.ru, online.sberbank.ru, googlead...",50,57,38,12,Galaxy A8 2018 Dual,Тосно,...,2,0,4,15,13,1,10,6,1,7
2,41.0,0,99002,"[castlots.org, googleads.g.doubleclick.net, i....",429,261,148,18,Honor 8X,Заречный,...,2,22,26,51,59,102,36,38,0,4
3,33.0,0,155506,"[tpc.googlesyndication.com, ad.mail.ru, m.avit...",9,0,29,0,iPhone 7 Plus,Краснодар,...,0,0,0,0,0,0,0,0,0,0
4,54.0,0,213873,"[yandex.ru, street-viewer.ru, yastatic.net, s0...",0,0,4,0,Honor 6A,Краснодар,...,0,0,0,0,0,0,0,0,0,0


In [57]:
df['is_male'].value_counts()

1    135331
0    128994
Name: is_male, dtype: int64

In [58]:
print(df.shape)
df.dropna(inplace=True)
print(df.shape)

(264325, 2553)
(264325, 2553)


In [59]:
feat = df.drop(columns=['user_id', 'age', 'is_male'])
tar = df['is_male']

In [60]:
feat['url_host'].head()

0    [sun9-29.userapi.com, id.tinkoff.ru, id.tinkof...
1    [onlayn-radio.ru, online.sberbank.ru, googlead...
2    [castlots.org, googleads.g.doubleclick.net, i....
3    [tpc.googlesyndication.com, ad.mail.ru, m.avit...
4    [yandex.ru, street-viewer.ru, yastatic.net, s0...
Name: url_host, dtype: object

In [61]:
%%time
feat['url_host'] = feat['url_host'].apply(url_transformer)

CPU times: total: 3.02 s
Wall time: 45.1 s


In [62]:
feat['url_host'].head()

0    esia.gosuslugi.ru sun9-5.userapi.com online.sb...
1    online.sberbank.ru m.auto.ru onlayn-radio.ru c...
2    cloud.mail.ru publication.pravo.gov.ru sun9-5....
3    instagram.com nmfo-spo.edu.rosminzdrav.ru goog...
4    yandex.ru street-viewer.ru s0.2mdn.net yastati...
Name: url_host, dtype: object

In [63]:
feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264325 entries, 0 to 264324
Columns: 2550 entries, url_host to night_7
dtypes: category(6), float32(1), int64(2535), int8(4), object(1), uint32(3)
memory usage: 5.0+ GB


In [64]:
feat['url_host'] = feat['url_host'].astype('U')

In [65]:
feat.head()

,url_host,request_cnt_day,request_cnt_evening,request_cnt_morning,request_cnt_night,cpe_model_name,city_name,region_name,cpe_type_cd,request_cnt_max_day,...,night_6,morning_6,evening_6,evening_5,morning_5,day_2,evening_7,day_5,night_1,night_7
0,esia.gosuslugi.ru sun9-5.userapi.com online.sb...,122,30,88,0,Redmi Note 7,Ижевск,Удмуртская Республика,smartphone,6,...,0,2,3,8,7,17,0,22,0,0
1,online.sberbank.ru m.auto.ru onlayn-radio.ru c...,50,57,38,12,Galaxy A8 2018 Dual,Тосно,Ленинградская область,smartphone,4,...,2,0,4,15,13,1,10,6,1,7
2,cloud.mail.ru publication.pravo.gov.ru sun9-5....,429,261,148,18,Honor 8X,Заречный,Пензенская область,smartphone,6,...,2,22,26,51,59,102,36,38,0,4
3,instagram.com nmfo-spo.edu.rosminzdrav.ru goog...,9,0,29,0,iPhone 7 Plus,Краснодар,Краснодарский край,smartphone,3,...,0,0,0,0,0,0,0,0,0,0
4,yandex.ru street-viewer.ru s0.2mdn.net yastati...,0,0,4,0,Honor 6A,Краснодар,Краснодарский край,smartphone,0,...,0,0,0,0,0,0,0,0,0,0


# Gender

In [66]:
ohe_encoder = OneHotEncoder(handle_unknown='ignore', drop='first')
oe_encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)

In [67]:
count_word = ColumnTransformer([('tf_idf', CountVectorizer(token_pattern=r'\b\S+\b'),'url_host')])
tf_idf_word = ColumnTransformer([('tf_idf', TfidfVectorizer(token_pattern=r'\b\S+\b'),'url_host')])
num_transformer_lr = ColumnTransformer(transformers=[('n', MinMaxScaler(), num_features_list+wd_list)])
cat_transformer_ohe = ColumnTransformer(transformers=[('c', ohe_encoder, cat_features_list)])
cat_transformer_oe = ColumnTransformer(transformers=[('c', oe_encoder, cat_features_list)])
top_2500_transformer = ColumnTransformer(transformers=[('n', FunctionTransformer(), top_2500)])
top_2500_normalizer = ColumnTransformer(transformers=[('ntop', Normalizer(norm='max'), top_2500)])
num_transformer_lgbm = ColumnTransformer(transformers=[('nft', FunctionTransformer(), num_features_list+wd_list)])

In [68]:
vectorizer_lr = FeatureUnion([('cat', cat_transformer_ohe), 
                              ('word', tf_idf_word), 
                              ('num', num_transformer_lr)])#,
                              #('t_2500', top_2500_normalizer)])

vectorizer_nb = FeatureUnion([('cat', cat_transformer_ohe), 
                              ('word', count_word)]) 

vectorizer_lgbm = FeatureUnion([('cat', cat_transformer_oe), 
                                ('top_2000', top_2500_transformer), 
                                ('num', num_transformer_lgbm)])

In [69]:
pipe_gender_lr = Pipeline([('transform', vectorizer_lr),
                           ('classifier',  LogisticRegression(max_iter=1000, solver='saga'))])

In [70]:
pipe_gender_nb = Pipeline([('transform', vectorizer_nb),
                           ('classifier', MultinomialNB())])

In [71]:
pipe_gender_lgbm = Pipeline([('transform', vectorizer_lgbm),
                             ('classifier', LGBMClassifier(n_estimators=2000))])

In [72]:
x_train, x_test, y_train, y_test = train_test_split(\
    feat, tar, test_size = 0.33, random_state = SPLIT_SEED)

In [73]:
%%time
pipe_gender_lr.fit(x_train, y_train);

CPU times: total: 40.7 s
Wall time: 46.6 s


In [74]:
print(gini(y_test, pipe_gender_lr.predict_proba(x_test)[:,1]))

0.7333501463423981


In [75]:
%%time
pipe_gender_nb.fit(x_train, y_train);

CPU times: total: 10.2 s
Wall time: 10.5 s


In [76]:
print(gini(y_test, pipe_gender_nb.predict_proba(x_test)[:,1]))

0.7110459392343447


In [77]:
%%time
pipe_gender_lgbm.fit(x_train, y_train);

CPU times: total: 12min 37s
Wall time: 1min 13s


In [78]:
print(gini(y_test, pipe_gender_lgbm.predict_proba(x_test)[:,1]))

0.7421897582227717


In [79]:
stacking_gender = StackingClassifier(estimators=[('1', pipe_gender_lr),
                                                 ('2', pipe_gender_nb),
                                                 ('3', pipe_gender_lgbm)],
                                     cv=3,
                                     n_jobs=2)

In [80]:
%%time
stacking_gender.fit(x_train, y_train);

CPU times: total: 6.11 s
Wall time: 6min 12s


In [81]:
print(gini(y_test, stacking_gender.predict_proba(x_test)[:,1]))

0.760961963317667


# Age

In [82]:
pipe_age_lr = Pipeline([('transform', vectorizer_lr),
                        ('classifier',  LogisticRegression())])

In [83]:
pipe_age_nb = Pipeline([('transform', vectorizer_nb),
                        ('classifier', MultinomialNB())])

In [84]:
pipe_age_lgbm = Pipeline([('transform', vectorizer_lgbm),
                          ('classifier', LGBMClassifier(n_estimators=1000))])

In [85]:
def age_bucket(x):
    return bisect.bisect_left([18,25,35,45,55,65], x)

In [86]:
tar_age = df['age'].map(age_bucket)

In [87]:
%%time
x_train, x_test, y_train, y_test = train_test_split(\
     feat, tar_age, test_size = 0.33, random_state = SPLIT_SEED)

CPU times: total: 4.27 s
Wall time: 24.9 s


In [88]:
%%time
pipe_age_lr.fit(x_train, y_train)
m.f1_score(y_test, pipe_age_lr.predict(x_test), average='weighted')

CPU times: total: 56.8 s
Wall time: 1min 6s


0.4247376422008774

In [89]:
%%time
pipe_age_nb.fit(x_train, y_train)
m.f1_score(y_test, pipe_age_nb.predict(x_test), average='weighted')

CPU times: total: 15.5 s
Wall time: 15.8 s


0.3900611637130776

In [90]:
%%time
pipe_age_lgbm.fit(x_train, y_train)
m.f1_score(y_test, pipe_age_lgbm.predict(x_test), average='weighted')

CPU times: total: 47min 8s
Wall time: 3min 35s


0.4457941449526746

In [91]:
stacking_age = StackingClassifier(estimators=[('1', pipe_age_lr),
                                              ('2', pipe_age_nb),
                                              ('3', pipe_age_lgbm)],
                                  final_estimator=CatBoostClassifier(iterations=2000, silent=True),
                                  cv=3, n_jobs=2)

In [92]:
%%time
stacking_age.fit(x_train, y_train)
print(m.classification_report(y_test, stacking_age.predict(x_test), \
                            target_names = ['<18', '18-25','25-34', '35-44', '45-54', '55-65', '65+']))

              precision    recall  f1-score   support

         <18       0.00      0.00      0.00       256
       18-25       0.54      0.46      0.50     10601
       25-34       0.53      0.61      0.57     28204
       35-44       0.43      0.52      0.47     25039
       45-54       0.40      0.27      0.32     13549
       55-65       0.42      0.32      0.36      7736
         65+       0.36      0.04      0.08      1843

    accuracy                           0.47     87228
   macro avg       0.38      0.32      0.33     87228
weighted avg       0.47      0.47      0.46     87228

CPU times: total: 4min 28s
Wall time: 15min 42s


In [93]:
m.f1_score(y_test, stacking_age.predict(x_test), average='weighted')

0.4621788827374467

# Submit gender

In [98]:
id_to_submit = pd.read_feather('D:\\ods_ai\\submission.feather')

In [99]:
%%time
stacking_gender.fit(feat, tar);

CPU times: total: 7.42 s
Wall time: 10min 25s


In [100]:
features = id_to_submit.merge(data_tf, how = 'inner', on = ['user_id'])

In [101]:
features = features.merge(add_features, how = 'inner', on = ['user_id'])

In [102]:
%%time
features['url_host'] = features['url_host'].apply(url_transformer)

CPU times: total: 78.1 ms
Wall time: 11.9 s


In [103]:
features['url_host'] = features['url_host'].astype('U')

In [104]:
features.drop(columns=['user_id'], inplace=True)

In [105]:
id_to_submit['is_male'] = stacking_gender.predict_proba(features)[:,1]

In [106]:
id_to_submit.head()

,user_id,is_male
0,221301,0.940997
1,31271,0.586812
2,211594,0.622290
3,253119,0.647237
4,192578,0.852762


# Submit age

In [107]:
%%time
stacking_age.fit(feat, tar_age);

CPU times: total: 5min 58s
Wall time: 23min 8s


In [108]:
id_to_submit['age'] = stacking_age.predict(features)

In [109]:
id_to_submit.head()

,user_id,is_male,age
0,221301,0.940997,1
1,31271,0.586812,3
2,211594,0.622290,2
3,253119,0.647237,3
4,192578,0.852762,2


In [110]:
id_to_submit.to_csv(f'{LOCAL_DATA_PATH}\\submission.csv', index = False)